Création d'un dataframe reprenant les deux fichiers de logs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_err = pd.read_csv('../Data/logEtl/241016_LogETLError.csv', sep=';', dtype={'Program_Id': str, 'Schedules_Id': str, 'Schedules_Name': str})

In [ ]:
df = pd.read_csv('../data/logEtl/241016_LogETL.csv',sep=';')

In [ ]:
df.columns

In [ ]:
df_filtered = df.drop(columns=['Server version', 'Client version'])

In [ ]:
df_filtered.columns

In [ ]:
df_err.columns

In [ ]:
df_err_filtered = df_err.drop(columns=['Server version', 'Client version'])

In [ ]:
#df_err_filtered.columns = [col + "_err" for col in df_err_filtered.columns]

In [ ]:
df_err_filtered.columns

In [ ]:
def categorize_message(message):
    message = message.lower()
    if "Guidez Atelier".lower() in message or "Guidez Social".lower()  in message:
        return "Guidez Atelier Error"
    elif "La taille du champ".lower()  in message:
        return "Field Size Error" 
    elif "Un incident est survenu Object reference not set to an instance of an object.".lower()  in message:
        return "Object Reference Error"
    elif "La valeur #REF!".lower()  in message:
        return "Value #REF! Error"
    elif "La valeur #VALUE!".lower()  in message:
        return "Value #VALUE! Error"
    elif "La valeur #N/A!".lower()  in message:
        return "Value #N/A! Error"
    elif "La valeur #DIV/0!".lower()  in message:
        return "Value #DIV/0! Error"
    elif "La valeur #NAME?".lower()  in message:
        return "Value #NAME? Error"
    elif "Connection Timeout Expired.  The timeout period elapsed during the post-login phase.".lower()  in message:
        return "Connection Timeout Error"
    elif "an error occurred during the pre-login handshake.".lower()  in message:
        return "Pre-Login Error"
    elif "Un incident est survenu L'instruction CREATE UNIQUE INDEX a été interrompue, car une clé dupliquée a été trouvée pour l'objet".lower()  in message:
        return "Unique Index Error"
    elif "Impossible de créer la table".lower()  in message:
        return "Create TABLE Error"
    elif "Un incident est survenu Échec de l'opération car un index ou des statistiques portant le nom".lower()  in message:
        return "Index Error (similar to unique index error)"
    elif "utilisé dans la clé primaire, sa valeur ne doit pas être vide".lower()  in message:
        return "Primary Key Error"
    elif "pas convertible en Heure".lower()  in message:
        return "Time Format Error"
    elif "Not a legal OleAut date.".lower()  in message or ("La date".lower()  in message and "est pas valide".lower()  in message):
        return "Date Format Error"
    elif "connexion au web service impossible".lower()  in message:
        return "Web Service Error"
    elif "impossible d'ouvrir la requête".lower()  in message and "Un incident est survenu".lower()  in message and "non valide".lower()  in message:
        return "SQL Invalid Query Error"
    elif "Impossible d\'ouvrir la requête SQL sur la connexion".lower()  in message and "Le délai d\'attente a été dépassé".lower()  in message:
        return "SQL Query Timeout Error"
    #elif "Le filtre de suppression n'est pas compatible SQL".lower()  in message:
    #    return "SQL Delete Filter Error"
    #elif "impossible d'ouvrir la requête sql".lower()  in message:
     #   return "SQL Other Error"
    elif "Le nom de colonne".lower()  in message and "n'existe pas dans la table ou la vue cible".lower()  in message:
        return "Column Name Error"
    else:
        return "Other Error"

df_err_filtered = df_err_filtered.assign(
    Message_Category=df_err_filtered["Message"].apply(categorize_message)
)

In [ ]:
colonnes_communes = list(set(df_filtered.columns) & set(df_err_filtered.columns))
print(colonnes_communes)

In [ ]:
# Ajouter les colonnes manquantes dans chaque DataFrame pour les aligner
for col in df_filtered.columns:
    if col not in df_err_filtered.columns:
        df_err_filtered[col] = None  # Ajout d'une colonne vide

for col in df_err_filtered.columns:
    if col not in df_filtered.columns:
        df_filtered[col] = None  # Ajout d'une colonne vide

# Concaténer les deux DataFrame tout en conservant la cohérence des colonnes
df_final = pd.concat([df_filtered, df_err_filtered], ignore_index=True)

df_final.head()  # Affiche les 5 premières lignes

In [ ]:
# Assurer que ETL_StartDateTime est bien au format datetime
df_final["ETL_StartDateTime"] = pd.to_datetime(df_final["ETL_StartDateTime"], errors='coerce')

# Trier le DataFrame par ETL_StartDateTime
df_final = df_final.sort_values(by="ETL_StartDateTime")

# Afficher les premières lignes pour vérifier
df_final.head()

In [ ]:
# Définir les bornes
date_min = "2024-08-24 23:32:03"
date_max = "2024-10-15 23:31:49"

# Filtrer les lignes entre les deux dates
df_borne = df_final[(df_final["ETL_StartDateTime"] >= date_min) & (df_final["ETL_StartDateTime"] <= date_max)]

# Afficher les premières lignes pour vérifier
df_borne.head()

In [ ]:
df_borne.to_csv("fusion_dataframe.csv", index=False)
print("Fichier exporté : fusion_dataframe.csv")

In [ ]:
df_borne.columns